In [12]:
!pip install transformers --quiet 
!pip install transformers[sentencepiece] --quiet
!pip install langchain --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
from transformers import pipeline
generator = pipeline(model="lmsys/vicuna-13b-v1.3")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [9]:
prompt = "1+1 equals to"

In [10]:
generator(prompt)

[{'generated_text': '1+1 equals to 2.\n\nIn the case of the 2D Is'}]

## 提示词模板

In [25]:
from langchain import PromptTemplate
 
template = """
I want you to act as a naming consultant for new companies.
What is a good name for a company that makes {product}?
"""
 
prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)
prompt = prompt.format(product="colorful socks")
generator(prompt, max_new_tokens = 100)

[{'generated_text': '\nI want you to act as a naming consultant for new companies.\nWhat is a good name for a company that makes colorful socks?\n\n1. Socktopus\n2. Rainbow Socks Co.\n3. Kaleidoscope Socks\n4. Sock Frenzy\n5. Sole Society\n6. Sock Hop\n7. Sock Swap\n8. Sock Studio\n9. Sock Symphony\n10. Sock Spectrum\n11. Sock Serenade\n12. Sock Sensation\n13. Sock Sensibility'}]

## 将少量提示词示例传递给提示模板，实现few shot examples

In [27]:
from langchain import PromptTemplate, FewShotPromptTemplate
 
# 首先，创建few shot示例
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]
 
# 接下来，我们指定模板来格式化我们提供的示例,为此使用 `PromptTemplate` 类
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)
 
# 最后，我们创建 `FewShotPromptTemplate` 对象
few_shot_prompt = FewShotPromptTemplate(
    # 这些是我们要插入到提示中的示例
    examples=examples,
    # 当我们将示例插入提示时，这是我们想要格式化示例的方式
    example_prompt=example_prompt,
    # 前缀是提示中示例之前的一些文本
    # 通常，这由指令组成。
    prefix="Give the antonym of every input",
    # 后缀是提示中示例后面的一些文本
    # 通常，这是用户输入的位置
    suffix="Word: {input}\nAntonym:",
    # 输入变量是整体提示所期望的变量
    input_variables=["input"],
    # example_separator 是我们用来将前缀、示例和后缀连接在一起的字符串
    example_separator="  ",
)
 
# 我们现在可以使用“format”方法生成提示
print(few_shot_prompt.format(input="big"))
prompt = few_shot_prompt.format(input="big")
generator(prompt, max_new_tokens = 100)

Give the antonym of every input  
Word: happy
Antonym: sad

  
Word: tall
Antonym: short

  Word: big
Antonym:


[{'generated_text': 'Give the antonym of every input  \nWord: happy\nAntonym: sad\n\n  \nWord: tall\nAntonym: short\n\n  Word: big\nAntonym: small\n\n  \nWord: fast\nAntonym: slow\n\n  \nWord: hot\nAntonym: cold\n\n  \nWord: light\nAntonym: dark\n\n  \nWord: easy\nAntonym: difficult\n\n  \nWord: young\nAntonym: old\n\n  \nWord: good\nAntonym: bad\n\n  \nWord: true\nAntonym: false\n\n  \nWord: new\nAntonym: old\n\n  '}]